# RSA

## Symmetrische Verschlüsselung
Die Verschlüsselung und die Entschlüsselung werden mit dem identischen Schlüssel durchgeführt.
Beispiel: Caesar-Schiffre --> Der Schlüssel ist beispielsweise 2 --> das bedeutet, dass alle Buchstaben im Alphabet um 2 Stellen verschoben werden.

## Asymmetrische Verschlüsselung
Es existiert ein private und ein public Key.
Beispiel: Wenn Alice eine Nachricht an Bob senden will, verschlüsselt sie ihre Nachricht mit dem public key von Bob und sendet die Nachricht. Jetzt kann nur Bob die Nachricht entschlüsseln, da nur er den private key hat.

In der Praxis werden die Verfahren kombiniert. Die eigentliche Übertragung der Nachrichten erfolgt symmetrisch. Für die Übertragung des Schlüssel wird ein Asymmetrisches Verfahren verwendet.

## So wird ein Schüsselpaar erzeugt
- Wähle zwei Primzahlen p und q aus
	- In der Praxis sind dies Zahlen mit ca. 1024 Bits (ca. 600 Dezimalstellen) und werden mit Primzahltests gesucht
- Bilde das Produkt n = pq
- Berechne phi von n = $\phi$(n) = (p-1)(q-1)  |  Also einfach p-1 mal q-1
- Ermittle eine Zahl e, die teilerfremd zu $\phi$(n) ist
- Berechne den Kehrwert d von e modulo $\phi$(n)
	- Das würde man mit dem euklidischen Algorithmus machen --> oder einfach e<sup>-1</sup> mod $\phi$(n)
- Fertig, wir haben e, d und n
	- e (encrypt) und n bilden den public-key
	- d (decrypt) und n bilden den private-key

Das ganze wird nur ein Mal gemacht und es können e, d und n immer wieder verwendet werden. Solange d geheim bleibt...

## So wird verschlüsselt und entschlüsselt
Nachrichten sind positive ganze Zahlen, die kleiner als n sind.
Eine Nachricht wird mit dem öffentlichen Schlüssel verschlüsselt.
S = N<sup>e</sup> mod n
Diese Berechnung würde in der Praxis mit sehr grossen Zahlen durchgeführt werden und muss daher sehr effizient gelöst werden. Ein binäre Exploitation kombiniert mit der modularen Arithmetik ist dabei essenziell.

Beispiel:

Wir wollen die Nachricht 52134 versenden.

N = 52134

S = 52134<sup>e</sup> mod n

S ist die verschlüsselte zu übermittelnde Nachricht.

Das Entschlüsseln funktioniert dann mit dem privaten Schlüssel:

N = S<sup>d</sup> mod n

### Wieso klappt das? Wieso gilt modulo n immer (N<sup>e</sup>)<sup>d</sup> = N?
Kurz gesagt:
Wenn N kein vielfaches von p ist, dann kann der kleine Satz von Fermat angewendet werden.
--> N<sup>p-1</sup> = 1
--> N<sup>ed</sup> = N

Wenn N ein vielfaches von p ist, also N = 0 in Z<sub>p</sub> und damit natürlich auch N<sup>ed</sup> = N

Wir haben somit ***p | N<sup>ed</sup> - N***  --> p ist ein Teiler der Differenz von N<sup>ed</sup> - N
Da p und q teilerfremd sind, folgt ***pq | N<sup>ed</sup> - N*** --> pq ist ein Teiler der Differenz von N<sup>ed</sup> - N

### Wie kann man den Code knacken?
- Kennt man p und q, so kann man sofort $\phi$(n) = (p-1)(q-1) berechnen
- Da e öffentlich ist, kann man dann auch d ermitteln
	- weil d der Kehrwert von e und phi ist --> d = e modulo $\phi$(n)
- Und so kommt man an den private key

Die Sicherheit des Verfahrens beruht also auf zwei "Erfahrungswerten":
- Ist n gross genug, so ist es schwer, n in Primfaktoren zu zerlegen
	- Es ist kein effizientes Verfahren bekannt, womit man eine grosse Zahl in seine Primfaktoren zerlegen kann
- Modulo n ist es schwer, N zu bestimmen, wenn man N<sup>e</sup> und e kennt
	- Auch hier gibt es kein bekanntes effizientes Verfahren

Es gibt aber keinen mathematischen Beweis dafür, dass dies ein sicheres Verschlüsselungsverfahren ist. Mit Quantencomputern kann das Verfahren geknackt werden, da es bereits einen Algorithmus gibt, der grosse Zahlen im seine Primfaktoren zerlegen kann.

## Weitere Anwendungsmöglichkeit
Alice könnte mit ihrem private key etwas verschlüsseln und an Bob senden. Bob kann dies mit dem public key von Alice entschlüsseln und mit der unverschlüsselten Version vergleichen und somit überprüfen, ob die Nachricht tatsächlich von Alice stammt.

In [1]:
# Beispiel RSA Schlüsselpaar erzeugen

p = 31
q = 47
n = p * q
phi = (p - 1) * (q - 1)
e = 7  # e muss teilerfremd zu phi sein
# das kleinste e kann einfach ermittelt werden --> phi / 2 rechnen, dann /3, etc. Bis das erste Mal eine Kommazahl rauskommt, dies ist dann eine teilerfremde Zahl
d = pow(e, -1, phi)  # d ist der Kehrwert von e
print(f'p: {p}, q: {q}, n: {n}, phi: {phi}, e: {e}, d: {d}')
print(f'Public Key: (e: {e}, n: {n})')
print(f'Private Key: (d: {d}, n: {n})')

p: 31, q: 47, n: 1457, phi: 1380, e: 7, d: 1183
Public Key: (e: 7, n: 1457)
Private Key: (d: 1183, n: 1457)


In [2]:
# Beispiel RSA Verschlüsselung und Entschlüsselung

N = 20
S = pow(N, e, n)  # Verschlüsselung

print(f'Original Nachricht: {N}')
print(f'Verschlüsselte Nachricht: {S}')

# Und hier entschlüsseln wir eine Nachricht
S = 10
N = pow(S, d, n)  # Entschlüsselung

print(f'Verschlüsselte Nachricht: {S}')
print(f'Original Nachricht: {N}')


Original Nachricht: 20
Verschlüsselte Nachricht: 731
Verschlüsselte Nachricht: 10
Original Nachricht: 1342


# Das ElGamal-Verfahren
Hier wird die Idee von Diffi-Hellman zur Verschlüsselung von Nachrichten verwendet.

1. A wählt eine Primzahl p und eine Primitivwurzel g modulo p
	1. 2 $\le$  g $\le$ p - 2. Es ist also g<sup>p-1</sup> = 1 (p)
2. Dann wählt A zufällig x $\in$ {1, 2, ..., p-2} und berechnet X = g<sup>x</sup> (p)
	1. Das Tripel (p, g, X) bildet A's öffentlicher Schlüssel und x ist ihr geheimer Schlüssel
3. Damit B eine Nachricht verschlüsselt an A senden kann, nimmt sie den öffentlichen Schüssel von A, wählt zufällig y $\in$ {1,2,..., p-2} und berechnet Y = g<sup>y</sup> (p)
4. B verschlüsselt den Klartext m $\in$ {0, 1, ..., p-1} zum Ciphertext c = X<sup>y</sup> * m (p) und sendet das Paar (Y, c) an A
5. A berechnet Y<sup>d</sup> * c (p) mit d = p -1 - x und erhält damit den Klartext m

Beispiel:
1. A wählt p = 7 und g = 3
2. A wählt x = 2 und berechnet X = 3<sup>2</sup> (p) = 2
	1. Der öffentliche Schlüssel von A ist das Tripel (7, 3, 2)
3. B wählt y = 4 und berechnet Y = 3<sup>4</sup> (p) = 4
4. B verschlüsselt den Klartext m = 6 zum Ciphertext c = 2<sup>4</sup> * m (p) = 16 * 6 (p) = 96 (p) = 5 und sendet das Paar (4, 5) an A
5. A berechnet d = 7 - 1 - 2 = 4 und
   4<sup>4</sup> * 5 (p) = 256 * 5 (p) = 1280 (p) = 6
6. Der Klartext ist also die 6!

In [3]:
#print Beispiel ElGamal privater Schlüssel vorhanden und jetzt öffentlichen bestimmen
p = 73
g = 7
x = 13

X = pow(g, x, p)

print(f'Public Key: (p: {p}, g: {g}, X: {X})')
print(f'Private Key: (x: {x})')

Public Key: (p: 73, g: 7, X: 66)
Private Key: (x: 13)


In [4]:
# Beispiel ElGamal Nachricht mit public key verschlüsseln
m = 4
y = 9

Y = pow(g, y, p)
c = (X ** y) * m % p

print(f'Ciphertext: (Y: {Y}, c: {c})')

Ciphertext: (Y: 10, c: 33)


In [5]:
# Beispiel ElGamal Nachricht mit private key entschlüsseln
Y = 56
c = 57
print(f'Ciphertext: (Y: {Y}, c: {c})')

d = p - 1 - x
m = (Y ** d) * c % p  # Auch möglich ist: (pow(Y, d, p) * c) % p

print(f'Klartext: {m}')

Ciphertext: (Y: 56, c: 57)
Klartext: 42
